In [41]:
import tensorflow as tf
from tensorflow.contrib import rnn

import pandas as pd
import numpy as np
import spacy
import seaborn as sns
import matplotlib.pyplot as plt
%matplotlib inline

nlp = spacy.load('en')

dtypes ={
    'id': np.uint32,
    'qid1': np.uint32,
    'qid2': np.uint32,
    'question1': np.str,
    'question2': np.str,
    'is_duplicate': np.uint8
}

df = pd.read_csv('../../dataset/quora_train.csv', dtype=dtypes, usecols=['question1', 'question2', 'is_duplicate'], nrows=1000)

In [77]:
def pad(v):
    zeros = np.zeros([100, 300])
    zeros[:v.shape[0],:v.shape[1]] = v
    return zeros
    
q1s = df['question1'].map(nlp).map(lambda x: np.matrix([t.vector for t in x])).map(pad)
q2s = df['question1'].map(nlp).map(lambda x: np.matrix([t.vector for t in x])).map(pad)

q1m = np.concatenate(q1s.values).flatten().reshape([1000, 100, 300])
q2m = np.concatenate(q2s.values).flatten().reshape([1000, 100, 300])

In [106]:
tf.reset_default_graph()

Q1 = tf.placeholder(tf.float32, shape=[None, 100, 300])
Q2 = tf.placeholder(tf.float32, shape=[None, 100, 300])

cell1 = rnn.BasicRNNCell(128)
cell2 = rnn.BasicRNNCell(128, reuse=True)
state = tf.get_variable(shape=[1000, 128], initializer=tf.zeros_initializer(), name='state')

outputs1, state = tf.nn.dynamic_rnn(cell1, Q1, initial_state=state)
outputs2, state = tf.nn.dynamic_rnn(cell2, Q2, initial_state=state)

In [118]:
sess = tf.Session()
sess.run(tf.global_variables_initializer())
print(sess.run(outputs1, feed_dict={Q1: q1m})[:,-1,:].shape)

(1000, 128)


In [123]:
outputs1

<tf.Tensor 'rnn/transpose:0' shape=(1000, 100, 128) dtype=float32>